In [45]:
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
train_df=pd.read_csv('Disaster_train.csv')
test_df=pd.read_csv('Disaster_test.csv')


In [48]:
train_df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [49]:
train_df.isnull().sum()

,0
id,0
keyword,61
location,2533
text,0
target,0


In [50]:
train_df['keyword']=train_df['keyword'].fillna('None')
train_df['location']=train_df['location'].fillna('Unknown')

In [51]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [52]:
# Function to clean text
def clean_text(text):
    text= text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    return text

In [57]:
# Clean the text
train_df['cleaned_text'] = train_df['text'].apply(clean_text)


In [60]:
import nltk

# ... your existing code ...

# Download 'punkt_tab' before using word_tokenize
nltk.download('punkt_tab')

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

# ... your existing code ...

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [62]:
# Function for stemming
def stem_text(text):
   words = word_tokenize(text)
   return ' '.join([stemmer.stem(word) for word in words])

train_df['cleaned_text'] = train_df['cleaned_text'].apply(stem_text)

In [63]:
# Function for lemmatization
def lemmatize_text(text):
   words = word_tokenize(text)
   return ' '.join([lemmatizer.lemmatize(word) for word in words])

# Apply lemmatization
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lemmatize_text)


In [64]:
# Tokenize the cleaned text for Word2Vec
tokenized_text = [word_tokenize(text) for text in train_df['cleaned_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
word2vec_model.save("word2vec.model")

In [65]:
# Function to get average Word2Vec vectors for each text
def get_vector(text):
    words = word_tokenize(text)
    vector = np.zeros(100)  # Assuming vector_size=100
    count = 0
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Apply the function to get vectors
train_df['text_vector'] = train_df['cleaned_text'].apply(get_vector)

Accuracy: 0.6999343401181878


Accuracy: 0.5659881812212738
